In [1]:
# read the api key
f = open("sec_api.key", "r")
api_key = f.readline()

In [2]:
import pandas as pd
# this is for us to toy with an already downloaded csv
df_10k = pd.read_csv('./TSLA/csv/10-K.csv')
print(df_10k['linkToFilingDetails'])

0     https://www.sec.gov/Archives/edgar/data/131860...
1     https://www.sec.gov/Archives/edgar/data/131860...
2     https://www.sec.gov/Archives/edgar/data/131860...
3     https://www.sec.gov/ix?doc=/Archives/edgar/dat...
4     https://www.sec.gov/Archives/edgar/data/131860...
5     https://www.sec.gov/Archives/edgar/data/131860...
6     https://www.sec.gov/Archives/edgar/data/131860...
7     https://www.sec.gov/Archives/edgar/data/131860...
8     https://www.sec.gov/Archives/edgar/data/131860...
9     https://www.sec.gov/Archives/edgar/data/131860...
10    https://www.sec.gov/Archives/edgar/data/131860...
11    https://www.sec.gov/Archives/edgar/data/131860...
12    https://www.sec.gov/Archives/edgar/data/131860...
Name: linkToFilingDetails, dtype: object


In [3]:
urls = df_10k[['ticker', 
                'formType', 
                'periodOfReport',
                'filedAt', 
                'linkToFilingDetails']].rename(columns={'linkToFilingDetails': 'filingUrl'})

# display(urls)

urls['financialReportsUrl'] = urls['filingUrl'].apply(lambda url: '/'.join(url.split('/')[:-1]) + '/Financial_Report.xlsx')

urls.head(2)

,ticker,formType,periodOfReport,filedAt,filingUrl,financialReportsUrl
0,TSLA,10-K,2022-12-31,2023-01-30T21:29:15-05:00,https://www.sec.gov/Archives/edgar/data/131860...,https://www.sec.gov/Archives/edgar/data/131860...
1,TSLA,10-K,2021-12-31,2022-02-04T20:11:27-05:00,https://www.sec.gov/Archives/edgar/data/131860...,https://www.sec.gov/Archives/edgar/data/131860...


In [4]:
import requests
for single_filing in urls.iloc:
    financial_report_path = single_filing['financialReportsUrl'].replace('https://www.sec.gov/Archives/edgar/data/', '')
    # sometimes they have a different format
    financial_report_path = financial_report_path.replace('https://www.sec.gov/ix?doc=/Archives/edgar/data/', '')

    # print(financial_report_path)
    base_url = 'https://archive.sec-api.io/' + financial_report_path

    render_api_url = base_url + '?token=' + api_key
    print(render_api_url)

    response = requests.get(render_api_url)

    file_name = f"{single_filing['ticker']}-{single_filing['periodOfReport']}-{single_filing['formType']}.xlsx"
    # print(file_name)
    output = open('./TSLA/spreadsheets/' + file_name, 'wb')
    output.write(response.content)
    output.close()


https://archive.sec-api.io/1318605/000095017023001409/Financial_Report.xlsx?token=6b15d4247a62add288a7595bee22a0178be6829426bc84601d34dbdc751f603e
https://archive.sec-api.io/1318605/000095017022000796/Financial_Report.xlsx?token=6b15d4247a62add288a7595bee22a0178be6829426bc84601d34dbdc751f603e
https://archive.sec-api.io/1318605/000156459021004599/Financial_Report.xlsx?token=6b15d4247a62add288a7595bee22a0178be6829426bc84601d34dbdc751f603e
https://archive.sec-api.io/1318605/000156459020004475/Financial_Report.xlsx?token=6b15d4247a62add288a7595bee22a0178be6829426bc84601d34dbdc751f603e
https://archive.sec-api.io/1318605/000156459019003165/Financial_Report.xlsx?token=6b15d4247a62add288a7595bee22a0178be6829426bc84601d34dbdc751f603e
https://archive.sec-api.io/1318605/000156459018002956/Financial_Report.xlsx?token=6b15d4247a62add288a7595bee22a0178be6829426bc84601d34dbdc751f603e
https://archive.sec-api.io/1318605/000156459017003118/Financial_Report.xlsx?token=6b15d4247a62add288a7595bee22a0178be6

In [5]:
import os
import pandas as pd
from zipfile import BadZipFile

directory = './TSLA/spreadsheets/'
file_names = os.listdir(directory)

excel_docs = []
for filename in file_names:
    path = os.path.join(directory, filename)
    try:
        excel_docs.append(pd.read_excel(path, sheet_name=None, engine='openpyxl'))
    except BadZipFile:
        print('Unable to read excel file from: ', path)

Unable to read excel file from:  ./TSLA/spreadsheets/TSLA-2010-12-31-10-K.xlsx
Unable to read excel file from:  ./TSLA/spreadsheets/TSLA-2011-12-31-10-K.xlsx
Unable to read excel file from:  ./TSLA/spreadsheets/TSLA-2012-12-31-10-K.xlsx


In [6]:
page = excel_docs[9]['Document and Entity Information']
# print(page.keys())

page = excel_docs[9]['Consolidated Statements of Rede']
# print(page.keys())

sheet_title = page.keys()[0]
column_names = []
for x in range(1, len(page.keys())):
    column_names.append(page.keys()[x])

print(column_names)

# page.iloc[0].iloc[0]




['Unnamed: 1', 'Total', 'Revision of Prior Period, Accounting Standards Update, Adjustment [Member]', 'Redeemable Noncontrolling Interests [Member]', 'Common Stock [Member]', 'Unnamed: 6', 'Additional Paid-In Capital [Member]', 'Additional Paid-In Capital [Member] Revision of Prior Period, Accounting Standards Update, Adjustment [Member]', 'Accumulated Other Comprehensive (Loss) Income [Member]', 'Accumulated Deficit [Member]', '[1]', 'Accumulated Deficit [Member] Revision of Prior Period, Accounting Standards Update, Adjustment [Member]', '[1].1', "Total Stockholder's Equity [Member]", "Total Stockholder's Equity [Member] Revision of Prior Period, Accounting Standards Update, Adjustment [Member]", 'Noncontrolling Interests in Subsidiaries [Member]']


In [50]:
from llama_index.llms.ollama import Ollama
from llama_index.experimental.query_engine import PandasQueryEngine

df = pd.DataFrame(
    {
        "city": ["Toronto", "Tokyo", "Berlin"],
        "population": [2930000, 13960000, 3645000],
    }
)
llm = Ollama(model='llama3', request_timeout=360.0, seed=42)
query_engine = PandasQueryEngine(df=df, verbose=True, llm=llm)


response = query_engine.query(
    "What is the city with the highest population?",
)

print(response)

> Pandas Instructions:
```
df.loc[df['population'].idxmax()]['city']
```
> Pandas Output: Tokyo
Tokyo


In [17]:
from llama_index.llms.ollama import Ollama
from llama_index.experimental.query_engine import PandasQueryEngine

page = excel_docs[9]['Digital Assets, Net']
print(page.keys())

llm = Ollama(model='llama3', request_timeout=360.0, seed=42)
query_engine = PandasQueryEngine(df=page, verbose=True, llm=llm)


response = query_engine.query(
    "Search every cell and list the cells with lots of text",
)

print(response)

Index(['Digital Assets, Net', '12 Months Ended'], dtype='object')
> Pandas Instructions:
```
Here is the Python expression that can be used to search every cell in the dataframe `df` and list the cells with lots of text:

len(df.applymap(len)) > 50
```
> Pandas Output: There was an error running the output as Python code. Error message: invalid syntax (<unknown>, line 1)
There was an error running the output as Python code. Error message: invalid syntax (<unknown>, line 1)


Traceback (most recent call last):
  File "C:\Users\liamd\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\llama_index\experimental\query_engine\pandas\output_parser.py", line 40, in default_output_processor
    tree = ast.parse(output)
           ^^^^^^^^^^^^^^^^^
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.12_3.12.1008.0_x64__qbz5n2kfra8p0\Lib\ast.py", line 52, in parse
    return compile(source, filename, mode, flags,
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<unknown>", line 1
    Here is the Python expression that can be used to search every cell in the dataframe `df` and list the cells with lots of text:
                ^^^^^^
SyntaxError: invalid syntax


In [20]:
print(type(page))

<class 'pandas.core.frame.DataFrame'>
